# Content-based Filtering

Aproach:
- Table with features erstellen, ratings in zweitem table

- sklearn-kNN mit minkowski-distance auf features anwenden, erhalte matrix mit allen Nachbarn eines jeden Films inklusive Abstand

- Wähle für jeden Nutzer und Film die x-nächsten Nachbarn aus, die der Nutzer bewertet hat

- Funktion schreiben, die auf basis des ratings der neighbours das rating eines Filmes vorhersagt.

In [1]:
import pandas as pd
import numpy as np
import string
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
kwargs = dict(random_state=42)

In [2]:
movies = pd.read_csv('../../data/preprocessed/movies_id_updated.csv')
average_ratings = pd.read_csv('../../data/preprocessed/average_ratings.csv')
omdb = pd.read_csv('../../data/preprocessed/omdb_cleaned.csv')
ratings = pd.read_csv('../../data/preprocessed/ratings_clean_std_0.csv')

## Feature Preprocessing

In [3]:
movies.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtPictureURL
0,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,tt0113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995.0,1068044-jumanji,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,tt0107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993.0,grumpy_old_men,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,tt0114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995.0,waiting_to_exhale,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,tt0113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995.0,father_of_the_bride_part_ii,http://content8.flixster.com/movie/25/54/25542...


In [4]:
ratings

,Unnamed: 0,user_id,imdbID,rating
0,0,1264,tt0047034,3.5
1,1,213,tt0304141,2.5
2,2,593,tt0369436,3.0
3,3,609,tt1077258,4.0
4,4,1590,tt0052182,4.0
...,...,...,...,...
787536,812812,1032,tt0083530,3.0
787537,812813,99,tt0107798,3.0
787538,812814,333,tt0093857,3.0
787539,812815,49,tt0144168,3.0


In [5]:
omdb

,0,Title,Year,Rated,Runtime,Writer,Plot,Language,imdbRating,imdbVotes,...,Released_month,Released_day,PG_Rating,Available_languages,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated,Unnamed: 0,0.1
0,tt0047034,Godzilla,1954.0,Not Rated,96.0,"Takeo Murata, Ishirô Honda, Shigeru Kayama (st...",American nuclear weapons testing results in th...,['Japanese'],7.6,27485.0,...,5.0,7.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN
1,tt0304141,Harry Potter and the Prisoner of Azkaban,2004.0,PG,142.0,"J.K. Rowling (novel), Steve Kloves (screenplay)","Harry Potter, Ron and Hermione return to Hogwa...",['English'],7.9,524511.0,...,6.0,4.0,1.0,1.0,0.0,2.0,0.0,0.0,NaN,NaN
2,tt0369436,Four Christmases,2008.0,PG-13,88.0,"Matt Allen (screenplay), Caleb Wilson (screenp...",A couple struggles to visit all four of their ...,['English'],5.7,61335.0,...,11.0,26.0,2.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN
3,tt1077258,Planet Terror,2007.0,Not Rated,105.0,Robert Rodriguez,"After an experimental bio-weapon is released, ...",['English'],7.1,196149.0,...,6.0,13.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN
4,tt0052182,Separate Tables,1958.0,Approved,100.0,"Terence Rattigan (screenplay), John Gay (scree...",The stories of several people are told as they...,['English'],7.4,6337.0,...,2.0,11.0,NaN,1.0,2.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9303,NaN,Pokemon 3: The Movie,2001.0,5.9,87.0,Satoshi Tajiri,"In the town of Greenfield, a young, lonely gir...",Japanese,NaN,12672.0,...,6.0,21.0,NaN,NaN,0.0,0.0,0.0,0.0,5533.0,tt0266860
9304,NaN,Pokemon 4Ever,2002.0,5.7,75.0,Hideki Sonoda,Ash must stop a hunter who forces the mythical...,Japanese,NaN,7600.0,...,7.0,7.0,NaN,NaN,0.0,0.0,0.0,0.0,5557.0,tt0313487
9305,NaN,Le chagrin et la pitie,1969.0,8.2,251.0,Marcel Ophuls,An in-depth exploration of the various reactio...,French,NaN,3484.0,...,9.0,18.0,NaN,NaN,0.0,0.0,0.0,0.0,6102.0,tt0383974
9306,NaN,Just the Ticket,1999.0,5.7,115.0,Richard Wenk,A ticket scalper wants to make one last big de...,English,NaN,1822.0,...,2.0,26.0,NaN,NaN,0.0,0.0,0.0,0.0,8060.0,tt0134948


In [6]:
omdb.columns

Index(['0', 'Title', 'Year', 'Rated', 'Runtime', 'Writer', 'Plot', 'Language',
       'imdbRating', 'imdbVotes', 'imdbID', 'Rotten Tomatoes', 'Metacritic',
       'Series', 'Released_season', 'Released_month', 'Released_day',
       'PG_Rating', 'Available_languages', 'Oscars_won', 'Oscars_nominated',
       'Golden_globe_won', 'Golden_globe_nominated', 'Unnamed: 0', '0.1'],
      dtype='object')

In [7]:
ratings

,Unnamed: 0,user_id,imdbID,rating
0,0,1264,tt0047034,3.5
1,1,213,tt0304141,2.5
2,2,593,tt0369436,3.0
3,3,609,tt1077258,4.0
4,4,1590,tt0052182,4.0
...,...,...,...,...
787536,812812,1032,tt0083530,3.0
787537,812813,99,tt0107798,3.0
787538,812814,333,tt0093857,3.0
787539,812815,49,tt0144168,3.0


In [11]:
 # Fill NaN Runtime
omdb.loc[:, 'Runtime'] = omdb.loc[:, 'Runtime'].fillna(omdb.loc[:, 'Runtime'].median())
# Fill NaN for imdbVotes
omdb.loc[:, 'imdbVotes'] = omdb.loc[:, 'imdbVotes'].fillna(omdb.loc[:, 'imdbVotes'].median())

9308

In [12]:
#entferne Ratings von den 10 Movies, für die Feature fehlen
ratings=ratings.merge(omdb[['imdbID', 'Runtime']], on='imdbID', how='left').dropna()

## KNN

In [13]:
# Standardize the features
scaler = preprocessing.StandardScaler()
features = scaler.fit_transform(features)

In [14]:
features

array([[-1.84817541, -0.37457714, -0.15494645, -0.08122467, -0.39494007],
       [ 0.86347469,  1.52406319,  4.02385213, -0.08122467,  1.63080103],
       [ 1.0804067 , -0.70477546,  0.129651  , -0.08122467, -0.39494007],
       ...,
       [-1.03468038,  6.02301527, -0.35673739, -0.08122467, -0.39494007],
       [ 0.59230968,  0.40964386, -0.37071083, -0.08122467, -0.39494007],
       [ 0.32114467, -0.20947798, -0.27879889, -0.08122467, -0.39494007]])

In [15]:
# features of the first entry (imdbID 439)
features[0]

array([-1.84817541, -0.37457714, -0.15494645, -0.08122467, -0.39494007])

In [16]:
features.shape[0]

9308

In [17]:
# create NearesNeighbor model with the minkowski distance
knn = NearestNeighbors(metric='minkowski', p=2, algorithm='brute', n_jobs=-1)

In [18]:
# fit to all the features
knn.fit(features)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# create two array for distance and index to store
distances = np.empty((features.shape[0],features.shape[0]))
indices = np.empty((features.shape[0],features.shape[0]))

# iterate over all imdbIDs
for i in range(0, features.shape[0]):
    # get all neighbors and the distances for each imdbID i
    distances[i], indices[i] = knn.kneighbors([features[i]], n_neighbors=features.shape[0])
    
    if i%1000==0:
        print(i)

In [ ]:
distances

In [ ]:
indices

In [ ]:
len(indices)

In [ ]:
distances_df = pd.DataFrame(distances)
distances_df

In [ ]:
indices_df = pd.DataFrame(indices)
indices_df

In [ ]:
#Alles in eine Funktion
def predict_rating(imdbID, userID):
    
    #Bestimme Index in omdb von der ImdbID
    index = omdb.loc[(omdb['imdbID'] == imdbID)].index

    #Speichere Reihenfolge der Indizes der Nachbarn des Films und tausche erste Reihe mit Spaltenbezeichnungen
    movie_row = indices_df.loc[index]
    movie_row = movie_row.reset_index(drop=True)
    movie_row.columns = movie_row.loc[0]
    movie_row.drop(0, inplace=True)
    movie_row.loc[1,:] = range(0,len(indices_df))
    
    #Speichere alle Filme, die der User bewertet hat und bestimme index davon
    rated_id = ratings.loc[ratings['user_id'] == userID]
    rated = np.empty((len(rated_id), 1))
    i = 0
    for imdb in rated_id['imdbID']:
        rated[i] = omdb.loc[(omdb['imdbID'] == imdb)].index
        i = i+1
    rated_df = pd.DataFrame(rated)
    
    # Bestimme die Position der bewerteten Filme innerhalb der Nachbarschaft
    position = movie_row[list(rated_df[0].astype(int))]
    position = position.transpose()
    position.columns = ['position']
    position ['Index'] = position.index
    position = position.reset_index(drop = True)
        
    #Greife auf den Film in der Distanzmatrix zu
    movie_row_distance = distances_df.loc[index]
    
    #Bestimme Distanz der Filme vom Film, die der User bewertet hat
    user_distances = movie_row_distance[list(position['position'].astype(int))].transpose()
    user_distances.columns = ['distance']
    user_distances =user_distances.reset_index(drop = True)
    
    #Füge alle Infos zusammen und sortiere von nächstem zum weitest entfernten Film
    neighbors = pd.concat([position, user_distances], axis=1, join = "inner")
    neighbors = neighbors.sort_values("distance")
    
    #Füge die Ratings des Users zu diesen Filmen noch hinzu
    rated_id['Index']= rated
    neighbors = neighbors.merge(rated_id[["rating","Index"]], on="Index", how = "outer")
    
    # Berechne die Prognose: Gewichtung der Bewertung der Nachbarn je nach Distanz
    pred = sum(neighbors['rating']*(1+(1-neighbors['distance']/neighbors['distance'].mean()))) / neighbors.shape[0]
    
    return pred

In [ ]:
predict_rating('tt0102467', 123)

In [ ]:
omdb

In [ ]:
predict_rating('tt0114709', 1228)

In [ ]:
n=100
test_set = ratings.sample(n)
test_set   

In [ ]:
predictions = np.empty(n)
reals = np.empty(n)
i = 0

for index, row in test_set.iterrows():
        imdbID = row['imdbID']
        userID = row['user_id']
        real = row['rating']
        pred = predict_rating(imdbID, userID)
        predictions[i] = pred
        reals[i] = real
        i = i+1
rmse = mean_squared_error(reals, predictions, squared=False)
print('RMSE: '+str(rmse))

In [ ]:
predict_rating('tt0114709', 1228)

In [ ]:
n=100
test_set = ratings.sample(n)
test_set   

In [ ]:
predictions = np.empty(n)
reals = np.empty(n)
i = 0

for index, row in test_set.iterrows():
        imdbID = row['imdbID']
        userID = row['user_id']
        real = row['rating']
        pred = predict_rating(imdbID, userID)
        predictions[i] = pred
        reals[i] = real
        i = i+1
rmse = mean_squared_error(reals, predictions, squared=False)
print('RMSE: '+str(rmse))